In [1]:
!pip install deep-translator


In [2]:
from deep_translator import GoogleTranslator
to_translate = 'עבודות עפר'
translated = GoogleTranslator(source='iw', target='en').translate(to_translate)
translated

'Earthworks'

In [3]:
from dotenv import load_dotenv
import os
local=True
load_dotenv(".env_generic")
model_file=".env_openai"  
load_dotenv(model_file)
print(os.getenv('OPENAI_MODEL_NAME'),os.getenv('OPENAI_API_BASE'))

gpt-4o https://api.openai.com/v1


In [4]:
import pandas as pd
xsls_doc_name="/Users/amit/Documents/IFC/gil_school.xlsx"

In [5]:
#pd.read_csv(doc_name)
df=pd.read_excel(xsls_doc_name).dropna(axis=1, how='all').dropna(axis=0, how='all')

In [6]:
import re
def get_compiled_pattern(pattern):
    return(re.compile(pattern))
    
compiled_section=get_compiled_pattern("^\d+$")
compiled_sub_section=get_compiled_pattern("^\d+.\d+$")
compiled_task=get_compiled_pattern("^\d+\.\d+\.\d+$")
compiled_item=get_compiled_pattern("^\d+\.\d+\.\d+\.\d+$")
    

In [7]:
import numpy as np
df["item_id"]=df["סעיף"].astype("str")
df["description"]=df["תאור"].astype("str")
df["row_type"]=df.apply(lambda x: "section" if compiled_section.match(x.item_id) else None, axis=1)
df["row_type"]=df.apply(lambda x: "sub_section" if compiled_sub_section.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "task" if compiled_task.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "item" if compiled_item.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "total_cost" if  "סה''כ" in x.description else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "redundant_title" if  "תאור" == x.description else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "comment" if x.row_type==None and x.description!="nan" else x.row_type, axis=1)


In [8]:
projects={'gil_school'}
section_list=[]
current_section=None
tc_1st =True
for index, row in df.iterrows():
    if (row['description']=='nan'):
        continue
    if(row['row_type']=="section"):
        if (current_section!=None):
            section_list.append(current_section)
        current_section={"section_name": row['description'], "section_id": row['item_id'], "sub_sections_list":[]}
    if(row['row_type']=="sub_section"):
        sub_section={"sub_section_name": row['description'], "sub_section_id": row['item_id'], "tasks_list":[]}
        current_section['sub_sections_list'].append(sub_section)
    if(row['row_type']=="task"):
        current_task={"task_name": row["description"] ,"task_id": row['item_id'], "items_list":[] }
        current_section['sub_sections_list'][-1]['tasks_list'].append(current_task)
    if(row['row_type']=="item"):
        current_item={"item_name": row["description"] ,"item_id": row['item_id'], "units":row["יח'"], "quantity": row["כמות"], "price": row["מחיר"], "item_cost": row['סה"כ']}
        current_section['sub_sections_list'][-1]['tasks_list'][-1]["items_list"].append(current_item)
    if(row['row_type']=="total_cost"):
        if tc_1st:
            current_section['sub_sections_list'][-1]['tasks_list'][-1]["task_cost"]=row['סה"כ']
            tc_1st=False
        else:
            current_section['sub_sections_list'][-1]["sub_section_cost"]=row['סה"כ']
            tc_1st=True
    if(row['row_type']=="comment"):
        current_section['sub_sections_list'][-1]["comment"]=row["description"]
        
#    print(row['row_type'], row['description'])
section_list
project={'sections': section_list}

In [9]:
import json 
with open("/Users/amit/Documents/IFC/gil_school.json", "w") as outfile: 
    json.dump(project, outfile)

In [10]:
exploded=pd.DataFrame.from_dict(project['sections']).explode('sub_sections_list')
exploded['tasks_list']=exploded['sub_sections_list'].apply(lambda x: x['tasks_list'])
exploded=exploded.explode('tasks_list')
exploded['items_list']=exploded['tasks_list'].apply(lambda x: x['items_list'])
exploded=exploded.explode('items_list')
exploded=exploded[~exploded['items_list'].isna()]
exploded['item_name']=exploded['items_list'].apply(lambda x: x['item_name'])
exploded['item_id']=exploded['items_list'].apply(lambda x: x['item_id'])
exploded['units']=exploded['items_list'].apply(lambda x: x['units'])
exploded['quantity']=exploded['items_list'].apply(lambda x: x['quantity'])
exploded['price']=exploded['items_list'].apply(lambda x: x['price'])
exploded['item_cost']=exploded['items_list'].apply(lambda x: x['item_cost'])
exploded['sub_section_name']=exploded['sub_sections_list'].apply(lambda x: x['sub_section_name'])
exploded['task_name']=exploded['tasks_list'].apply(lambda x: x['task_name'])

In [11]:
exploded[['item_id','section_name','sub_section_name','task_name', 'item_name', 'units', 'quantity', 'price', 'item_cost']]
for field in ['section_name','sub_section_name','task_name']:
    print (field)
    exploded[f"translated_{field}"]=exploded[field].apply(lambda x: GoogleTranslator(source='iw', target='en').translate(x))

section_name
sub_section_name
task_name


In [12]:
exploded.columns

Index(['section_name', 'section_id', 'sub_sections_list', 'tasks_list',
       'items_list', 'item_name', 'item_id', 'units', 'quantity', 'price',
       'item_cost', 'sub_section_name', 'task_name', 'translated_section_name',
       'translated_sub_section_name', 'translated_task_name'],
      dtype='object')

In [13]:
exploded[[f"translated_section_name", "section_name"]]
for field in ['section_name','sub_section_name','task_name']:
    print (f"translated_{field}")
    exploded[f"translated_{field}"]=exploded[f"translated_{field}"].apply(lambda x: x.replace("-","").strip())
    print (exploded[f"translated_{field}"])

translated_section_name
0     School building
0     School building
0     School building
0     School building
0     School building
           ...       
5    Storey balconies
5    Storey balconies
5    Storey balconies
5    Storey balconies
5    Storey balconies
Name: translated_section_name, Length: 1302, dtype: object
translated_sub_section_name
0                     Earthworks
0                     Earthworks
0    Cast concrete works on site
0    Cast concrete works on site
0    Cast concrete works on site
                ...             
5                    development
5                    development
5                    development
5                    development
5                    development
Name: translated_sub_section_name, Length: 1302, dtype: object
translated_task_name
0                     Earthworks
0                     Earthworks
0    Cast concrete works on site
0    Cast concrete works on site
0    Cast concrete works on site
                ...             
5 

In [14]:
items_df=exploded[['item_id','translated_section_name','translated_sub_section_name','translated_task_name', 'item_name', 'units', 'quantity', 'price', 'item_cost']]
items_df['item_cost']=items_df['item_cost'].apply(lambda x:x if str(x).isnumeric() else 0)

/var/folders/10/4kpf7wcs453gwy7fx3rdzhh40000gn/T/ipykernel_28754/3021263839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_df['item_cost']=items_df['item_cost'].apply(lambda x:x if str(x).isnumeric() else 0)


In [15]:
items_df=items_df.rename(columns={"item_id":"id","embeddings":"values" })
items_df['metadata']=items_df.apply(lambda x: {"project_name": "gil_school","section_name":x.translated_section_name , "sub_section_name":x.translated_sub_section_name,\
                          "task_name":x.translated_task_name, "units": x.units, "quantity":x.quantity, "price": x.price, "item_cost":x.item_cost }, axis=1)

In [16]:


items_df['enriched_item']=items_df.apply(lambda x: x.item_name + ". measured by: " + x.units +". quantity: " + str(x.quantity) +\
                                         ". price: " + str(x.price) + ". item_cost: " + str(x.item_cost), axis=1) 

items_df['metadata']=items_df.apply(lambda x: {"project_name": "gil_school","id": x.id,"section_name":x.translated_section_name , "sub_section_name":x.translated_sub_section_name,\
                          "task_name":x.translated_task_name, "units": x.units, "quantity":x.quantity, "price": x.price, "item_cost":x.item_cost }, axis=1)

In [17]:
from uuid import uuid4

from langchain_core.documents import Document
documents=[]
for index, row in items_df[['enriched_item', 'metadata']].iterrows():
    doc= Document(
        page_content=row["enriched_item"],
        metadata=row["metadata"],
    )
    documents.append(doc)

In [18]:
len(documents)

1302

In [19]:
uuids = [str(uuid4()) for _ in range(len(documents))]

In [20]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# configure client
pc = Pinecone(api_key=pinecone_api_key)
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
index_name = 'quantities-qa-openai'
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

vector_store.add_documents(documents=documents, ids=uuids)

['62444a23-f6c3-493e-85e9-cd3db447d0cd',
 'd6b802c0-b0f4-43be-8d43-26acfcf59852',
 'bdceecbe-c311-46af-8fe4-136c1676dc97',
 '98b137dc-a120-40c4-9a1c-e775f975259f',
 'a4d29393-ff00-4667-b560-34fc6cc737f8',
 '799be669-c543-480c-9afd-5768c7167f85',
 '8d9559c9-3900-49d6-89df-fc7090278bf7',
 'd31d333b-5354-4181-bffa-dcc79779ff10',
 '74501eb0-372b-4d04-bb15-7d694e1424c3',
 '04a7b01e-c2a3-4e67-9a6f-7ef4626718a0',
 'b9fec8e5-481b-46c9-8531-c4acf8b0bfb8',
 '311bbdb4-15e7-41d5-9aba-17c0269c19bb',
 '0c020d36-4989-4aad-bb92-c1d802ee37f4',
 'c950d54f-1aa9-4584-a828-10fa2f53b77a',
 '5acb39a2-5e35-4e10-b0c5-4558b38b3b88',
 '723bb899-c749-4733-87a8-dec944fb0b1b',
 '550d023a-be66-45c5-8f46-3323672f51de',
 '1efe43b9-35b3-4c89-ad51-6ebd558dc3a7',
 '3a10431e-6caf-4f3b-95ea-6ef7892801f2',
 '74eff665-d044-4314-8b17-c8959a28880f',
 '94edf6f1-8b7c-4d0f-9136-bd103759bbe5',
 '5550fce7-ada0-409f-8032-fb59e07543f7',
 'f1d49e7f-38e4-4ddd-8d68-51949c8d408a',
 '74ded128-cd4d-4283-ac85-cfd8bb33c2ce',
 '8d03c97c-16b7-

In [21]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# configure client
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'quantities-qa-openai'
index = pc.Index(index_name)
#embed_model = "text-embedding-ada-002"

In [ ]:

query = (
    "all"
)

xq = embeddings.embed_query(query)

# retrieve from Pinecone
#xq = res['data'][0]['embedding']
#'section_name': 'פיתוח\xa0נופי-קומת\xa0קרקע',
#                           'sub_section_name': 'עבודות\xa0פיתוח',
# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=300,include_metadata=True, 
    filter={
        "item_cost": {"$gt": 14800.0},
    })

In [ ]:
res

In [ ]:
section_name_set=set()
sub_section_name_set=set()
task_name_set=set()
for i in (res['matches']):
    section_name_set.add(i['metadata']['section_name'])
    sub_section_name_set.add(i['metadata']['sub_section_name'])
    task_name_set.add(i['metadata']['task_name'])

In [ ]:
    filter={
        "price": {"$gt": 14800.0},
        "section_name": {"$eq": "פיתוח\xa0נופי-קומת\xa0קרקע"},
        "sub_section_name": {"$eq": "עבודות\xa0פיתוח"}
    })